In [1]:
import pandas as pd
import numpy as np

import nltk
import re
import csv

import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm

import bq_helper
from bq_helper import BigQueryHelper

import warnings
warnings.filterwarnings('ignore', message=r'Label not .* is present in all training examples.')

pd.set_option('display.max_colwidth', 300)

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

from sklearn.metrics import f1_score, accuracy_score

In [3]:
stack_overflow = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                         dataset_name="stackoverflow")

Using Kaggle's public dataset BigQuery integration.


In [4]:
bg = BigQueryHelper("bigquery-public-data", "stackoverflow")
bg.list_tables()

Using Kaggle's public dataset BigQuery integration.


['badges',
 'comments',
 'post_history',
 'post_links',
 'posts_answers',
 'posts_moderator_nomination',
 'posts_orphaned_tag_wiki',
 'posts_privilege_wiki',
 'posts_questions',
 'posts_tag_wiki',
 'posts_tag_wiki_excerpt',
 'posts_wiki_placeholder',
 'stackoverflow_posts',
 'tags',
 'users',
 'votes']

In [5]:
bg.head("stackoverflow_posts", num_rows=1)

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,525063,Android Respond To URL in Intent,<p>I want my intent to be launched when the user goes to a certain url: for example the android market does this with <a href= http://market.android.com/ >http://market.android.com/</a> urls. so does youtube. I want mine to do that too.</p>,525086,1,2,None,2009-02-08 02:55:56.867000+00:00,65,2016-04-10 03:28:09.653000+00:00,2015-08-15 10:24:12.720000+00:00,None,1529129,Isaac Waller,764272,None,1,132,android|url|android-intent|intentfilter|launch,91060


In [6]:
bg.table_schema("stackoverflow_posts")

,name,type,mode,description
0,id,INTEGER,REQUIRED,None
1,title,STRING,NULLABLE,None
2,body,STRING,NULLABLE,None
3,accepted_answer_id,INTEGER,NULLABLE,None
4,answer_count,INTEGER,NULLABLE,None
5,comment_count,INTEGER,NULLABLE,None
6,community_owned_date,TIMESTAMP,NULLABLE,None
7,creation_date,TIMESTAMP,NULLABLE,None
8,favorite_count,INTEGER,NULLABLE,None
9,last_activity_date,TIMESTAMP,NULLABLE,None


In [7]:
query = """
        SELECT 
            id, title , tags 
        FROM 
            `bigquery-public-data.stackoverflow.stackoverflow_posts`
        WHERE
            title NOT LIKE '%None%' AND 
            (tags LIKE '%|python|%' OR tags LIKE '%|c#|%' OR
            tags LIKE '%|java|%' OR tags LIKE '%|r|%' OR
            tags LIKE '%|android|%' OR tags LIKE '%|html|%' OR
            tags LIKE '%|c++|%' OR tags LIKE '%|sql|%' OR
            tags LIKE '%|c|%' OR tags LIKE '%kotlin%') AND 
            LENGTH(tags) < 20
        LIMIT
             10000;
        """

data = stack_overflow.query_to_pandas(query)

data_copy = data.copy()

data.head()

,id,title,tags
0,32824619,How to add a Map to RequestBody instead of adding key value pairs individually when using okhttp?,java|android|okhttp
1,19823353,HTML: order of divs and a tags for href,css|html|href
2,344317,Where does gcc look for C and C++ header files?,c++|c|gcc|header
3,22883935,MySQL Search/Replace with a variable number,mysql|sql|regex
4,698509,Nothing is extracted from the SQlite table?,sql|c|sqlite


In [8]:
data.title = data.title.str.replace('<[^<]+?>','')
data.title = data.title.str.replace('http','')
data.title = data.title.str.replace('[^\w\s]','')
data.title = data.title.str.lower()
data.head()

,id,title,tags
0,32824619,how to add a map to requestbody instead of adding key value pairs individually when using ok,java|android|okhttp
1,19823353,html order of divs and a tags for href,css|html|href
2,344317,where does gcc look for c and c header files,c++|c|gcc|header
3,22883935,mysql searchreplace with a variable number,mysql|sql|regex
4,698509,nothing is extracted from the sqlite table,sql|c|sqlite


In [9]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

data['title'] = data['title'].apply(lambda x: remove_stopwords(x))
data.head()

,id,title,tags
0,32824619,add map requestbody instead adding key value pairs individually using ok,java|android|okhttp
1,19823353,html order divs tags href,css|html|href
2,344317,gcc look c c header files,c++|c|gcc|header
3,22883935,mysql searchreplace variable number,mysql|sql|regex
4,698509,nothing extracted sqlite table,sql|c|sqlite


In [10]:
data['tags'] = data['tags'].str.split('|')
data.head()

,id,title,tags
0,32824619,add map requestbody instead adding key value pairs individually using ok,"[java, android, okhttp]"
1,19823353,html order divs tags href,"[css, html, href]"
2,344317,gcc look c c header files,"[c++, c, gcc, header]"
3,22883935,mysql searchreplace variable number,"[mysql, sql, regex]"
4,698509,nothing extracted sqlite table,"[sql, c, sqlite]"


In [11]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(data['tags'])

y = multilabel_binarizer.transform(data['tags'])

In [12]:
x_train, x_val, ytrain, yval = train_test_split(data['title'],
                                                y, test_size=0.2)

In [13]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.5, min_df=1, max_features=5000)
xtrain = tfidf_vectorizer.fit_transform(x_train)
xval = tfidf_vectorizer.transform(x_val)

In [14]:
lr = LogisticRegression()
classifier = OneVsRestClassifier(lr)
classifier.fit(xtrain, ytrain)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='warn',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='warn', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [15]:
y_pred = classifier.predict(xval)
f1_score(yval, y_pred, average="micro"), accuracy_score(yval, y_pred)

(0.5335441608312628, 0.058)

In [16]:
pred_prob = classifier.predict_proba(xval)
t = 0.3
predp = (pred_prob >= t).astype(int)
f1_score(yval, predp, average="micro"), accuracy_score(yval, predp)

(0.60713589393644, 0.103)

In [17]:
def predict(m):
    m = remove_stopwords(m)
    m_vec = tfidf_vectorizer.transform([m])
    pred_prob = classifier.predict_proba(m_vec)
    t = 0.3
    predp = (pred_prob >= t).astype(int)
    #m_pred = classifier.predict(m_vec)
    return multilabel_binarizer.inverse_transform(predp)

In [18]:
for i in range(10):
    k = x_val.sample(1).index[0]
    print("Title: ", data_copy['title'][k],
          "\nPredicted tags: ", predict(x_val[k])),
    print("Actual tags: ",data['tags'][k], "\n")

Title:  PHP Syntax Error Nav Menu 
Predicted tags:  [('html', 'php')]
Actual tags:  ['php', 'html', 'css'] 

Title:  Saving html data in two different mysql table with single submit button 
Predicted tags:  [('html', 'mysql', 'php', 'sql')]
Actual tags:  ['php', 'html', 'mysql'] 

Title:  MySQL Is that a good practise to have a separate column with year of date in order to increase performance? 
Predicted tags:  [('mysql', 'sql')]
Actual tags:  ['mysql', 'sql', 'group'] 

Title:  Our client doesn't like pop-ups what should we do? 
Predicted tags:  [('html',)]
Actual tags:  ['jquery', 'html', 'css'] 

Title:  How to update separate counters dynamically 
Predicted tags:  [('html', 'php')]
Actual tags:  ['java', 'android', 'xml'] 

Title:  Need an example on how to get preferred language from Accept-Language request header 
Predicted tags:  [('html', 'java')]
Actual tags:  ['c#', 'java', 'php', '.net'] 

Title:  Connect JAVA with SQLEXPRESS 
Predicted tags:  [('android', 'java')]
Actual t